<center style="font-size:2em;font-weight:bold">
   The Minimum Height of the Center of Mass
</center>
<p/>
<center style="font-size:1.75em;font-weight:bold">
   of a Right-Cylindrical Container of Liquid
</center>
<p/>
<center>Thomas E. Vaughan</center>

<!--- vim: set filetype=markdown tw=75: -->



# Introduction

In the shower one morning, I groggily reached for the shampoo, but I ended
up knocking the container onto its side.  I was surprised by my clumsiness
and woke up a bit.  I saw that there are two main reasons for the ease with
which I had knocked over the container.  One was obvious and uninteresting,
but the other was interesting enough to keep me thinking long after I
exited the shower.

The obvious, uninteresting reason is that the container was almost empty.
When a container has minimum content, its inertia is minimal.  Any
external, disturbing force has maximal effect on an object of minimal
inertia.

Yet, with water dripping off of my face and onto the floor, I stared down
at the container of shampoo as it lay sideways on the floor of the shower.
In that moment of thought, I realized that the low mass and low rotational
inertia of the nearly empty container did not satisfyingly explain what had
happened.  My disturbance of the container had been only very slight
indeed.  Even in its low-mass state, the container had not suffered much of
an angular perturbation before it fell over.  There was something more
interesting involved as well.

<!--- vim: set filetype=markdown tw=75: -->

# Configuration of Notebook

The following javascript-magic cell enables the editing of any cell in the
external editor, gvim.

<!--- vim: set filetype=markdown tw=75 -->



In [1]:
%%javascript

// This cell defines two command-mode shortcuts.
//
// Pressing 'g' in command mode copies the content of the current cell into
// a file and then launches gvim to edit that file.
//
// Pressing 'u' in command mode updates the current cell with the contents
// of the file.
//
// The idea is first to press 'g', then to edit the file with gvim, then to
// exit from gvim, and finally to press 'u' to update the cell from the
// file.

// The 'g' shortcut.
IPython.keyboard_manager.command_shortcuts.add_shortcut('g', {
    handler : function (event) {
        var input = IPython.notebook.get_selected_cell().get_text();
        var cmd = "f = open('.toto.txt', 'w');f.close()";
        if (input != "") {
            cmd = '%%writefile .toto.txt\n' + input;
        }
        IPython.notebook.kernel.execute(cmd);
        cmd = "import os;"
        // Establish a clean PATH that has only operating-system-native
        // executables.  On my machine, having '/opt/miniconda3/bin' in the
        // PATH, as it is by default, causes youcompleteme to malfunction
        // because it finds '/opt/miniconda3/bin/python' before
        // '/usr/bin/python'.
        cmd = cmd + "os.putenv('PATH','/bin:/usr/bin');"
        cmd = cmd + "os.system('gvim .toto.txt')";
        IPython.notebook.kernel.execute(cmd);
        return false;
    }}
);

// The 'u' shortcut.
IPython.keyboard_manager.command_shortcuts.add_shortcut('u', {
    handler : function (event) {
        function handle_output(msg) {
            var ret = msg.content.text;
            IPython.notebook.get_selected_cell().set_text(ret);
        }
        var callback = {'output': handle_output};
        var cmd = "f = open('.toto.txt', 'r');print(f.read())";
        IPython.notebook.kernel.execute(cmd, {iopub: callback},
                                        {silent: false});
        return false;
    }}
);

// vim: set filetype=javascript tw=75 sw=4:



<IPython.core.display.Javascript object>

The following cell brings in various python libraries.

In [2]:
from IPython         import display
from IPython.display import SVG
from ipywidgets      import *

import matplotlib.pyplot as plt
import numpy as np
import svgwrite as sw

%matplotlib inline

# vim: set tw=75:



# Stability Against Angular Displacement

The container's stability against falling over when released from rest at
an angular displacement about the edge of the base depends on the height of
the contained liquid.  Before I finished with my shower, I had convinced
myself that both the completely empty state and the completely full state
are the least stable states.  Each has the highest center of mass, about
half the height of the container.

This can easily be seen via a couple of visualizations of the model
implemented in the python class `container`.

<!--- vim: set filetype=markdown tw=75: -->



In [3]:
class container:
    """
    Model of a cylindric container of liquid.

    After construction, the vertical coordinate of the center of mass is
    available in the field 'com'.

    The coordinate system is a bit odd, though.  The origin is at the
    upper-left corner of the drawing, and positive vertical coordinates are
    below that corner.

    The various methods pull out the geometry of the drawable components in
    a way that is easy to plot via the svgwrite module.
    """

    def __init__(self, wh, th, bw, lh, wd):
        """
        Initialize parameters.
        wh -- wall height
        th -- wall thickness
        bw -- base width
        lh -- liquid height
        wd -- wall density
        """
        self.wh = wh
        self.th = th
        self.bw = bw
        self.lh = lh
        self.wd = wd
        # Calculate height of center of mass.
        r1  = 0.5*bw - th            # inner radius of container
        r2  = 0.5*bw                 # outer radius of container
        ld  = 1                      # liquid density
        wm  = wd*(r2*r2 - r1*r1)*wh  # proportional to mass of wall
        lm  = ld*(r1*r1        )*lh  # proportional to mass of liquid
        self.com = wh - (0.5*wh*wm + 0.5*lh*lm)/(wm + lm)

    def left_wall(self):
        """ Upper-left corner and size of rectangle for left wall. """
        return ((0, 0), (self.th, self.wh))

    def right_wall(self):
        """ Upper-left corner and size of rectangle for right wall. """
        return ((self.bw - self.th, 0), (self.th, self.wh))

    def liquid(self):
        """ Upper-left corner and size of rectangle for liquid. """
        lw = self.bw - 2.0*self.th
        return ((self.th, self.wh - self.lh), (lw, self.wh))

    def base(self):
        """ Beginning point and ending point of line for base. """
        return ((0, self.wh), (self.bw, self.wh))

    def center_line(self):
        """
        Beginning point and ending point of line for center of mass.
        """
        return ((0, self.com), (self.bw, self.com))

    def tip_line(self):
        """
        Beginning point and ending point of line from edge of base to
        center of mass.
        """
        return ((0, self.wh), (0.5*self.bw, self.com))

    def tip_angle_loc(self):
        """
        Coordinates of location for drawing character representing tip angle.
        """
        return (0.15*self.bw,  self.com + 0.3*(self.wh - self.com))

    def tip_angle(self):
        """ Maximum tip-angle (degrees) before falling over. """
        # Note that denominator has to correct for weird coordinate system.
        return np.arctan(0.5*self.bw/(self.wh - self.com))*180.0/np.pi

# vim: set filetype=python tw=75:



The first visualization superimposes onto a schematic drawing of the
container and of the liquid surface both a line representing the height of
the center of mass and a line that shows the maximum, stable tip-angle.
Sliders allow the user to modify the parameters of the model and to get
instant, visual feedback.

<!--- vim: set filetype=markdown tw=75: -->



In [4]:
def drawing(wall_thck, wall_dnst, base_wdth, liqd_hght):
    """
    Use the svgwrite module to draw schematic of container of liquid.

    Parameters:
    wall_thck -- Relative to wall height, thickness of walls.
    base_wdth -- Relative to wall height, width of base.
    liqd_hght -- Relative to wall height, height of liquid surface.
    wall_dnst -- Relative to liquid density, density of walls.
    """

    sz = 500 # Size (pixels) of drawing.
    dr = sw.Drawing("drawing.svg", (sz, sz))
    th = wall_thck*sz # absolute wall thickness
    bw = base_wdth*sz # absolute base width
    lh = liqd_hght*sz # absolute liquid height
    wh = sz           # absolute wall height
    c  = container(wh, th, bw, lh, wall_dnst)

    dr.add(dr.rect(*c.left_wall() , fill  ='black'))
    dr.add(dr.rect(*c.right_wall(), fill  ='black'))
    dr.add(dr.rect(*c.liquid()    , fill  ='blue' ))
    dr.add(dr.line(*c.base()      , stroke='black'))

    dr.add(dr.line(*c.center_line(), stroke='green'  , stroke_width=3))
    dr.add(dr.line(*c.tip_line()   , stroke='magenta', stroke_width=3))
    dr.add(dr.text('α', insert=c.tip_angle_loc(), stroke='magenta',
                   fill='magenta', font_size=24))

    # Return SVG object.
    return SVG(dr.tostring())

interact(drawing,
         wall_thck=(0.0, 0.05, 0.001),
         base_wdth=(0.0, 1.00, 0.010),
         liqd_hght=(0.0, 1.00, 0.010),
         wall_dnst=(0.0, 1.00, 0.010))

# vim: set filetype=python tw=75:



interactive(children=(FloatSlider(value=0.025, description='wall_thck', max=0.05, step=0.001), FloatSlider(val…

<function __main__.drawing(wall_thck, wall_dnst, base_wdth, liqd_hght)>

The threshold angle $\alpha$ at which the container would fall over because
of gravity is the inverse tangent of the ratio of two lengths,

1. $\frac{L}{2}$, half the length of the base, and
2. $h$, the height of the center of mass.

\begin{equation}
   \alpha = \arctan \frac{L}{2h}
\end{equation}

But $h$ has its maximum possible value, half the height $H$ of the
container, both when the container is completely empty and when it is
completely full.
\begin{equation}
   h_{\text{max}} = \frac{H}{2}
\end{equation}
The threshold tipping angle is minimized in these conditions.
\begin{equation}
   \alpha_{\text{min}} = \arctan \frac{L}{4H}
\end{equation}
So, somewhere in between completely emtpy and completely full, there must
be a liquid level that gives the container the maximum safe tipping angle,
$\alpha_{\text{max}}$.  This corresponds to the minimum height
$h_{\text{min}}$ of the center of mass.

But how might I compute and expression for $h_{\text{min}}$?

<!--- vim: set tw=75: -->

In [5]:
x = np.linspace(0, 2*np.pi)

def update(omega = 1.0):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(x, np.sin(omega*x))
    fig.canvas.draw()

interact(update)

interactive(children=(FloatSlider(value=1.0, description='omega', max=3.0, min=-1.0), Output()), _dom_classes=…

<function __main__.update(omega=1.0)>

The following cell brings in the libraries needed for linear regression
and for plotting.

In [6]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline